In [1]:
import numpy as np
import uniprot
import pprint
import urllib.request ,urllib.error
from bs4 import BeautifulSoup
import pandas as pd
from pandas.core.frame import DataFrame
import requests
from bs4 import BeautifulSoup
from pyfaidx import Fasta
import csv
import shutil
import os

In [18]:
# Get data type by input the matrix id
def get_data_type(matrix_id):
    url_of_detailed_inf = "https://jaspar.genereg.net/matrix/"
    url_matrix = f"{url_of_detailed_inf}{matrix_id}/"
    response = requests.get(url_matrix)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="table")
    if table:
        rows = table.find_all("tr")
        columns = rows[7].find_all("td")
        HS_data_type = columns[1].get_text().strip()
    
    return HS_data_type

In [19]:
# Get uniprot id by input the matrix id
def get_uniprot_id(matrix_id):
    url_of_detailed_inf = "https://jaspar.genereg.net/matrix/"
    url_matrix = f"{url_of_detailed_inf}{matrix_id}/"
    response = requests.get(url_matrix)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", class_="table")
    if table:
        rows = table.find_all("tr")
        columns = rows[9].find_all("td")
        uniprot_id = columns[1].get_text().strip()

    return uniprot_id

In [20]:
# get the information
def get_inf(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    id_name_pairs = []

    table = soup.find("table", class_="table")
    if table:
        rows = table.find_all("tr")

        for row in rows[1:]:
            columns = row.find_all("td")
            if len(columns) >= 2:
                HS_id = columns[1].get_text().strip()
                HS_name = columns[2].get_text().strip()
                HS_class = columns[4].get_text().strip()
                HS_Family = columns[5].get_text().strip()

                HS_data_type = get_data_type(HS_id)
                HS_uniprot_id = get_uniprot_id(HS_id)
                
                id_name_pairs.append((HS_id, HS_name, HS_class,HS_Family,HS_data_type,HS_uniprot_id))
    
    return id_name_pairs

# matrix id, class, family, data type

In [ ]:
# Get the DNA id of the Homo sapiens
base_url = "https://jaspar.genereg.net/search?q=Homo%20sapiens&collection=CORE&tax_group=vertebrates"
total_inf = []

page = 1

while page <= 95:
    print(page)
    page_inf = []
    page_url = f"{base_url}&page={page}"
    page_inf = get_inf(page_url)

    total_inf.extend(page_inf)
    page += 1
    

#for HS_id, HS_name,HS_class,HS_Family,HS_data_type,HS_uniprot_id in total_inf:
#    print(f"ID: {HS_id}, Name: {HS_name}, Class:{HS_class}, Family:{HS_Family}, Data type:{HS_data_type}, Uniprot ID:{HS_uniprot_id}")

In [ ]:
# store the above data to .csv

file_path = "DNA_inf.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Name","Class","Family","Data type", "Uniprot ID"]) 
    
    for item in total_inf:
        writer.writerow(item)

In [21]:
def get_protein_sequence(uniprot_id):
    baseUrl = "http://www.uniprot.org/uniprot/"
    uniprot_url = baseUrl + uniprot_id + ".fasta"

    response = requests.post(uniprot_url)
    protein_Data=''.join(response.text)

    return protein_Data

In [22]:
def store_fasta(uniprot_id,lines,sequence):
    file_path = f"{uniprot_id}.fasta"
    
    with open(file_path, mode='w') as file:
        file.write(lines[0] + " ")
        file.write(sequence)
        
    # move to the folder
    source_file_path = f"{uniprot_id}.fasta" 
    destination_folder = "FASTA_protein File"  
    shutil.move(source_file_path, destination_folder)

In [2]:
DNA_inf = pd.read_csv("success_download.csv")
matrix_ID_list = []

for i in range(len(DNA_inf)):
    matrix_ID_list.append(DNA_inf.loc[i,"ID"])

In [3]:
DNA_inf

,ID,Name,Class,Family,Data type,Uniprot ID
0,MA0002.1,RUNX1,Runt domain factors,Runt-related factors,SELEX,Q01196
1,MA0003.2,TFAP2A,Basic helix-span-helix factors (bHSH),AP-2,ChIP-seq,P05549
2,MA0003.4,TFAP2A,Basic helix-span-helix factors (bHSH),AP-2,ChIP-seq,P05549
3,MA0007.2,AR,Nuclear receptors with C4 zinc fingers,Steroid hormone receptors (NR3),ChIP-seq,P10275
4,MA0014.2,PAX5,Paired box factors,Paired domain only,ChIP-seq,Q02548
...,...,...,...,...,...,...
325,MA1973.1,ZKSCAN3,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q9BRR0
326,MA1974.1,ZNF211,C2H2 zinc finger factors,Factors with multiple dispersed zinc fingers,NaN,Q13398
327,MA1979.1,ZNF416,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q8NA42
328,MA1983.1,ZNF582,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q96NG8


In [ ]:
# get the sequence from the inputting ID
id_list = []
for uniprot_id in uniprot_ID_list:
    print(uniprot_id)
    
    if uniprot_id in id_list:
        print("already printed!!!!!!!!!!")
    elif os.path.isfile(os.path.join("FASTA_protein File", f"{uniprot_id}.fasta")):
        print("already in file")
    else:
        protein_Data = get_protein_sequence(uniprot_id)
        
        lines = protein_Data.split('\n')
        sequence = ''.join(lines[1:])
        store_fasta(uniprot_id,lines,sequence)

        id_list.append(uniprot_id)

In [ ]:
import requests
import os

baseUrl = "https://jaspar.genereg.net/download/data/2022/sites/"
folder_path = "FASTA_matrix File" 

for matrix in matrix_ID_list:
    
    matrix_url = baseUrl + matrix + ".sites"
    file_name = f"{matrix}.fasta"

    file_path = os.path.join(folder_path, file_name)
    response = requests.get(matrix_url)

    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"File '{file_name}' successed")
    else:
        print(f"File '{file_name}' failed")
        failed_file.append(matrix)